In [19]:
import tkinter as tk
from tkinter import messagebox, ttk
import joblib
import numpy as np
from catboost import CatBoostRegressor

# Load the CatBoost models or create new instances if needed
model_parlier_path = 'C:\\Users\\asus1\\Desktop\\catb1.joblib'
model_meloland_path = 'C:\\Users\\asus1\\Desktop\\catb2.joblib'

try:
    # Attempt to load the pre-trained models
    model_parlier = joblib.load(model_parlier_path)
    model_meloland = joblib.load(model_meloland_path)
except FileNotFoundError:
    # If the models aren't found, initialize new CatBoostRegressor instances
    model_parlier = CatBoostRegressor()  # Ensure the models are trained beforehand
    model_meloland = CatBoostRegressor()

# Labels for input fields (only X1 and X4 now)
labels = {
    'X1: Average air temperature (℃)': 'T (℃)',
    'X4: Net radiation (W/m²)': 'Rn (W/m²)'
}

# Define the GUI application with two tabs
class PredictionApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Predicting Reference Evapotranspiration")
        self.geometry("800x500")
        
        # Create notebook for tabs with custom style
        style = ttk.Style(self)
        style.configure("TNotebook.Tab", font=("Consolas", 14, "bold"), padding=[20, 10])
        style.map("TNotebook.Tab", background=[("selected", "#a6e3e9")], foreground=[("selected", "blue")])

        # Create notebook for tabs
        notebook = ttk.Notebook(self)
        notebook.pack(fill="both", expand=True)

        # Create two tabs using tk.Frame instead of ttk.Frame for background color customization
        self.tab_parlier = tk.Frame(notebook, bg="#f2f7f7")
        self.tab_meloland = tk.Frame(notebook, bg="#f0f7ff")
        
        notebook.add(self.tab_parlier, text="Parlier (Stn. 39)")
        notebook.add(self.tab_meloland, text="Meloland (Stn. 87)")

        # Create widgets for both tabs
        self.create_widgets(self.tab_parlier, model_parlier, "Parlier (Stn. 39)", (2.6, 34.1), (-22, 194), bg_color="#f2f7f7")
        self.create_widgets(self.tab_meloland, model_meloland, "Meloland (Stn. 87)", (4.8, 39.1), (-1, 187), bg_color="#f0f7ff")

    def create_widgets(self, tab, model, station_name, x1_range, x4_range, bg_color):
        font_size = 12  # Control the font size here

        # Title Section
        title_label = tk.Label(tab, text=f"Predicting Reference Evapotranspiration\nfor Regions with Climate Conditions Similar to {station_name}", font=("Consolas", font_size + 4, "bold"), bg=bg_color)
        title_label.pack(pady=12)

        # Input Section
        input_frame = tk.LabelFrame(tab, text="Input", padx=10, pady=10, font=("Consolas", font_size, "bold"), bg=bg_color)
        input_frame.pack(padx=10, pady=10, fill="x")

        # Create entry widgets dynamically for each input field (only X1 and X4)
        self.entries = {}
        idx = 0
        for key, label_text in labels.items():
            label = tk.Label(input_frame, text=key, font=("Consolas", font_size, "italic"), bg=bg_color)
            label.grid(row=idx, column=0, sticky="w")
            entry = tk.Entry(input_frame, font=("Consolas", font_size))
            entry.grid(row=idx, column=1)
            self.entries[key] = entry

            # Display the valid range for each input
            if key == 'X1: Average air temperature (℃)':
                range_label = tk.Label(input_frame, text=f"Range: {x1_range[0]} to {x1_range[1]} ℃", font=("Consolas", font_size - 2, "italic"), bg=bg_color)
            elif key == 'X4: Net radiation (W/m²)':
                range_label = tk.Label(input_frame, text=f"Range: {x4_range[0]} to {x4_range[1]} W/m²", font=("Consolas", font_size - 2, "italic"), bg=bg_color)
            range_label.grid(row=idx, column=2, padx=5)
            idx += 1

        # Prediction Section
        prediction_frame = tk.LabelFrame(tab, text="Prediction", padx=10, pady=10, font=("Consolas", font_size, "bold"), bg=bg_color)
        prediction_frame.pack(padx=10, pady=10, fill="x")

        self.prediction_label = tk.Label(prediction_frame, text="ETo (mm): Reference Evapotranspiration = ", font=("Consolas", font_size, "italic"), bg=bg_color)
        self.prediction_label.grid(row=0, column=0, sticky="w")

        self.prediction_result = tk.Label(prediction_frame, text="", font=("Consolas", font_size), bg=bg_color)
        self.prediction_result.grid(row=0, column=1, sticky="w")

        # Predict Button
        predict_button = tk.Button(prediction_frame, text="Predict", command=lambda: self.predict(model, x1_range, x4_range), font=("Consolas", font_size))
        predict_button.grid(row=1, column=0, pady=5)

        # Clear Button
        clear_button = tk.Button(prediction_frame, text="Clear", command=self.clear, font=("Consolas", font_size))
        clear_button.grid(row=1, column=1, pady=5)

    # Prediction function with input validation (only X1 and X4)
    def predict(self, model, x1_range, x4_range):
        try:
            # Collect inputs from the user (only X1 and X4)
            inputs = []
            for key in labels:
                value = self.entries[key].get().strip()
                if value == "":
                    raise ValueError(f"Please enter a valid number for {key}.")
                # Ensure the input is a valid float (allow decimals) and within the valid range
                try:
                    value = float(value)
                    if key == 'X1: Average air temperature (℃)':
                        min_val, max_val = x1_range
                    elif key == 'X4: Net radiation (W/m²)':
                        min_val, max_val = x4_range
                    if not (min_val <= value <= max_val):
                        raise ValueError(f"Input for {key} is out of range ({min_val} to {max_val}).")
                    inputs.append(value)
                except ValueError as ve:
                    raise ValueError(f"Invalid input for {key}: {ve}")
            
            # Convert inputs to the correct format for prediction
            inputs = np.array(inputs).reshape(1, -1)
            # Predict the output using the model
            prediction = model.predict(inputs)
            # Display the predicted value (ETo in mm/day)
            self.prediction_result.config(text=f"{prediction[0]:.2f} mm/day")
        except Exception as e:
            messagebox.showerror("Error", str(e))

    # Clear function
    def clear(self):
        # Clear all input fields and prediction result
        for entry in self.entries.values():
            entry.delete(0, tk.END)
        self.prediction_result.config(text="")

# Run the application
if __name__ == "__main__":
    app = PredictionApp()
    app.mainloop()
